In [1]:
import awkward as ak
import dask
import dask_awkward
from distributed import performance_report
import hist
from hist.dask import Hist
import json
from coffea import processor
from coffea.nanoevents import BaseSchema, PFNanoAODSchema, NanoAODSchema
from coffea.dataset_tools import apply_to_dataset, apply_to_fileset, preprocess
from coffea.dataset_tools import max_chunks, max_files, slice_chunks, slice_files
from coffea.analysis_tools import Weights, PackedSelection
import matplotlib.pyplot as plt
from coffea.nanoevents.methods import vector
from coffea.nanoevents import NanoEventsFactory

/tmp/ipykernel_674384/3704368154.py:14: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [2]:

import fsspec_xrootd


In [3]:
# import uproot
# url = "root://cmsxrootd.fnal.gov//store/group/lpcpfnano/jdickins/v2_2/2018/GluGluHToBB/GluGluHToBB_Pt-200ToInf_M-125_TuneCP5_MINLO_13TeV-powheg-pythia8/GluGluHToBB/220322_131235/0000/nano_mc2018_20.root"
# uproot.open(url)["Events"].show()

In [4]:
# import XRootD.client


In [5]:
from msdprocessor import msdProcessor

In [6]:
from dask.distributed import Client
# client = Client("tls://localhost:8786")

client = Client
client

distributed.client.Client

In [7]:
# with open("2018_GluGluHToBB.json", "rt") as file:
#     initial_fileset = json.load(file)

In [8]:
with open("2018_GluGluHToBB.json", "rt") as file:
    initial_fileset = json.load(file)

In [9]:
files={}
for i, j in initial_fileset.items():
    print(f"this is i {i}")
    files[i]={"files":{}}
    for k in j:
        # k = k.replace("cmsxrootd.fnal.gov", "xcache")
        files[i]["files"][k] = "Events"
        print(f"this is k {k}")

        # files[i]["files"][k] = "Events"

this is i GluGluHToBB
this is k root://cmsxrootd.fnal.gov//store/group/lpcpfnano/jdickins/v2_2/2018/GluGluHToBB/GluGluHToBB_Pt-200ToInf_M-125_TuneCP5_MINLO_13TeV-powheg-pythia8/GluGluHToBB/220322_131235/0000/nano_mc2018_1-1.root
this is k root://cmsxrootd.fnal.gov//store/group/lpcpfnano/jdickins/v2_2/2018/GluGluHToBB/GluGluHToBB_Pt-200ToInf_M-125_TuneCP5_MINLO_13TeV-powheg-pythia8/GluGluHToBB/220322_131235/0000/nano_mc2018_1.root
this is k root://cmsxrootd.fnal.gov//store/group/lpcpfnano/jdickins/v2_2/2018/GluGluHToBB/GluGluHToBB_Pt-200ToInf_M-125_TuneCP5_MINLO_13TeV-powheg-pythia8/GluGluHToBB/220322_131235/0000/nano_mc2018_10.root
this is k root://cmsxrootd.fnal.gov//store/group/lpcpfnano/jdickins/v2_2/2018/GluGluHToBB/GluGluHToBB_Pt-200ToInf_M-125_TuneCP5_MINLO_13TeV-powheg-pythia8/GluGluHToBB/220322_131235/0000/nano_mc2018_11.root
this is k root://cmsxrootd.fnal.gov//store/group/lpcpfnano/jdickins/v2_2/2018/GluGluHToBB/GluGluHToBB_Pt-200ToInf_M-125_TuneCP5_MINLO_13TeV-powheg-pythia8

In [10]:
# print(files)

In [11]:
# output = processor.run_uproot_job(files=files, 
#                        treename="Events",
#                        processor_instance=Processor(),
#                        executor=processor.dask_executor,
#                        executor_args={'client': client, 'schema': processor.NanoAODSchema},
#                        chunksize=50000)

In [12]:
preprocessed_available, preprocessed_total = preprocess(
        files,
        step_size=30_000,
        align_clusters=None,
        skip_bad_files=True,
        recalculate_steps=False,
        files_per_batch=1,
        file_exceptions=(OSError,),
        save_form=True,
        uproot_options={},
        step_size_safety_factor=0.5,
    )

In [13]:
import gzip, pickle, json
output_file = "scaleout_fileset"
with gzip.open(f"{output_file}_available.json.gz", "wt") as file:
    json.dump(preprocessed_available, file, indent=2)
    print(f"Saved available fileset chunks to {output_file}_available.json.gz")
with gzip.open(f"{output_file}_all.json.gz", "wt") as file:
    json.dump(preprocessed_total, file, indent=2)
    print(f"Saved complete fileset chunks to {output_file}_all.json.gz")

Saved available fileset chunks to scaleout_fileset_available.json.gz
Saved complete fileset chunks to scaleout_fileset_all.json.gz


In [14]:
test_preprocessed_files = max_files(preprocessed_available, 30)
test_preprocessed = max_chunks(test_preprocessed_files, 30)

In [15]:
# test_preprocessed

In [16]:
compute, results = apply_to_fileset(data_manipulation=msdProcessor(),    
                            fileset=test_preprocessed,
                            schemaclass=PFNanoAODSchema,
                            uproot_options={"allow_read_errors_with_report": (OSError, KeyError)},
                           )

#--------------------------------------------------------------------------
#                         FastJet release 3.4.1
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code,
# CGAL and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------


In [18]:
print(type(results))

<class 'dict'>


In [20]:
compute = dask.compute(compute)

ValueError: spans must have compatible lengths

In [ ]:
result_compute = dask.compute(result)

In [21]:
print(result_compute)


NameError: name 'result_compute' is not defined

In [ ]:
tg_result = dask.compute(tg)
rep_result = dask.compute(rep)